In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_path = './drive/My Drive/Colab Notebooks/Speech Denoising/Data/'

In [ ]:
!pip install librosa

In [ ]:
import librosa

In [ ]:
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
s, sr=librosa.load(base_path + 'train_clean_male.wav', sr=None)
train_clean=librosa.stft(s, n_fft=1024, hop_length=512).T

sn, sr=librosa.load(base_path + 'train_dirty_male.wav', sr=None)
train_dirty=librosa.stft(sn, n_fft=1024, hop_length=512).T

In [ ]:
train_clean.shape, train_dirty.shape

### Take magnitudes of audio signals

In [ ]:
train_clean_val = np.abs(train_clean)
train_dirty_val = np.abs(train_dirty)

Check bounds of absolute values

In [ ]:
print(f'min(train_clean_val) = {np.min(train_clean_val)}, max(train_clean_val) = {np.max(train_clean_val)}')
print(f'min(train_dirty_val) = {np.min(train_dirty_val)}, max(train_dirty_val) = {np.max(train_dirty_val)}')

## Training

### Split the data into training and validation set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_dirty_val, train_clean_val, test_size=0.2)

### Define tensorflow variables for the model


In [ ]:
X = tf.placeholder(dtype='float', shape=[None, 513])
y = tf.placeholder(dtype='float', shape=[None, 513])

## Prediction

In [ ]:
def clean_audio(filename):
    # load audio file
    s, sr = librosa.load(filename, sr=None)
    
    # apply short time fourier transform
    dirty = librosa.stft(s, n_fft=1024, hop_length=512).T
    
    # take magnitudes of audio signals
    dirty_val = np.abs(dirty)
    
    # run it through the neural network to remove noisy bits
    clean_val = sess.run(output, feed_dict={X: dirty_val})
    
    # recover speech spectogram of the cleaned signal
    cleaned = np.multiply(np.divide(dirty.T, dirty_val.T), clean_val.T)
    return cleaned

In [ ]:
def save_audio(cleaned, to_filename):
    # recover time domain speech signal by applying inverse short time fourier transform
    sh_test = librosa.istft(cleaned, hop_length=512)
    
    # Save to a file
    librosa.output.write_wav(to_filename, sh_test, sr)
    print('Saved to ', to_filename)

In [ ]:
cleaned = clean_audio(base_path + 'test_x_01.wav')
save_audio(cleaned, base_path + 'test_x_01_cleaned.wav')

In [ ]:
cleaned = clean_audio(base_path + 'test_x_02.wav')
save_audio(cleaned, base_path + 'test_x_02_cleaned.wav')

In [ ]:
sess.close()